<a href="https://colab.research.google.com/github/DanielWarfield1/common_core_example_explorer/blob/main/2024_10_03_HardOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Okie Dokie
here's the sitch wade

I made https://github.com/DanielWarfield1/common_core_example_explorer which currently only supports hard coded queries and responses via a mock API, so I'm re-structuring this to create properly structured data so that I can put it in that mock API.

In [ ]:
!pip install groundx-python-sdk --upgrade
!pip install langchain-community langchain-core langchain_openai
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.5/358.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.1
    Uninstalling cryptography-43.0.1:
      Successfully uninstalled cryptography-43.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
bucket_id = 11644

In [ ]:
from pprint import pprint
from groundx import Groundx
from google.colab import userdata

groundx = Groundx(
    api_key=userdata.get('GroundXAPIKey_daniel.warfield')
)

In [ ]:
from pprint import pprint
from groundx import Groundx

print(f'{bucket_id} {type(bucket_id)}')

response = groundx.search.content(
    id=bucket_id,
    query="I need examples of calculating area"
)

11644 <class 'int'>


In [ ]:
relevent_docs = set([])
for res in response.body['search']['results']:

    relevent_docs.add(res['sourceUrl'])
for d in relevent_docs:
    print(d)

https://upload.groundx.ai/file/hsa.sse.a.2_unit%207_lesson%2012.pdf
https://upload.groundx.ai/file/7.g.a.1_nwea.pdf
https://upload.groundx.ai/file/5.md.c.5.c_nwea.pdf
https://upload.groundx.ai/file/2.md.b.5_nwea.pdf
https://upload.groundx.ai/file/gr%206.p.5%20quilt%20triangles_final.pdf
https://upload.groundx.ai/file/4.md.a.3_parcc.pdf
https://upload.groundx.ai/file/5.md.a.1_sbac.pdf
https://upload.groundx.ai/file/6.g.a.4_parcc.pdf
https://upload.groundx.ai/file/gr%202.p.1%20place%20value%20units_final%20.pdf
https://upload.groundx.ai/file/5.nf.b%20multiplication%20and%20division.pdf
https://upload.groundx.ai/file/3.md.c.7d_sbac.pdf
https://upload.groundx.ai/file/7.g.b.4_nwea.pdf
https://upload.groundx.ai/file/5_kde_measurement_and_data_volume_grade_5.pdf
https://upload.groundx.ai/file/gr%205.p.2%20forest%20road_final.pdf
https://upload.groundx.ai/file/3.md.d.8_nwea.pdf
https://upload.groundx.ai/file/6.g.a.4_nwea.pdf


# Update
while GX is fair at providing relevent examples, I think employing an LLM would be helpful. in outputing more relvent output. So, like RAG, but the user doesn't actually see the LLM response.

to make this happen I think I need to first take GXs output and turn it into a slightly different structure which I can feed to an LLM for reranking.

Also I noticed that

In [ ]:
retreival_data = []

query="give me good math problems for 2nd graders"

response = groundx.search.content(
    id=bucket_id,
    query=query
)

for res in response.body['search']['results']:
    suggested_text = res['suggestedText']

    document_data = groundx.documents.get(document_id = res['documentId']).body
    organizational_data = document_data['document']['searchData']
    pdf_url = document_data['document']['fileName'].replace(' ','%20')

    retreival_data.append({'description_of_document': suggested_text, 'metadata': organizational_data, 'url': pdf_url})

# retreival_data

In [ ]:
import openai
openai.api_key = userdata.get('OpenAIAPIKey')

In [ ]:
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Describe up to five examples which are most relevent to the query. The output should be a list of objects which include the \"url\", \"metadata\", and a brief textual description of why it's relevent, called \"description\". Make the output in a json format. Do not include markdown formatting around the json object. Only include good examples. If less than five examples are good, you may output fewer."},
        {"role": "user", "content": f"query: {query}\n\n=====example list to be ranked======\n\n{retreival_data}"}
    ]
)
llm_response = response.choices[0].message.content.replace('```json','').replace('```','')

In [ ]:
import json

print(f'query: {query}\n\nresponse:')

json.loads(llm_response)

query: give me good math problems for 2nd graders

response:


[{'url': 'http://achievethecore.org/content/upload/2.MD.C.8_NWEA.pdf',
  'metadata': {'example_index': 60, 'type': 'main example'},
  'description': "This example provides a second-grade math assessment problem involving money and calculations with coins. It is designed to enhance students' understanding of counting and calculating total amounts using different denominations, aligning with the Common Core standard for Measurement and Data. This is highly relevant for second graders as it involves basic arithmetic involving real-life scenarios with coins."},
 {'url': 'http://achievethecore.org/content/upload/Gr%202.P.2%20Two%20snakes_Final%20.pdf',
  'metadata': {'example_index': 85,
   'link_instance_index': 0,
   'link_type_index': 0,
   'type': 'linked example'},
  'description': 'This document offers a Grade 2 math problem that involves using basic addition and subtraction to solve a word problem comparing the lengths of two snakes. The task aligns with the Common Core standard by r

# futsing with google search to find the exploration


In [ ]:
import urllib.request, json
with urllib.request.urlopen("https://achievethecore.org/standards-admin/json.php") as url:
    data = url.read().decode()
website_data = json.loads(data.replace('window.cc = ',''))

In [ ]:
!pip install googlesearch-python

In [ ]:

from googlesearch import search
from random import randint
import urllib.parse

def google_search(query):

    #constructing an "I'm feeling lucky" google search URL
    query = urllib.parse.quote('archive the core coherence map ' + query)
    return f'http://www.google.com/search?q={query}&btnI'

    # Perform the Google search and get the top result
    search_results = search(query, num_results=1)

    # Return the first result link
    for result in search_results:
        return result

# putting some of the text fields in google search tends to consistently retreive the cohenrence map which is relevent to the main example
# query = "archivethecore coherence map <p>Determine an explicit expression, a recursive process, or steps for calculation from a context.</p>\r\n"
# query = "archivethecore coherence map <p>Understand a <a id=A number expressible in the form a/b where a is a whole number and b is a positive whole number. (The word fraction in these standards always refers to a non-negative number.) See also: rational number."
# first_link = google_search(query)
# first_link

In [ ]:
# document_data['document']['searchData']['example_index']

In [ ]:
# example_desc = website_data['standards'][str(document_data['document']['searchData']['example_index'])]['desc']
# query = f'archivethecore coherence map {example_desc}'
# first_link = google_search(query)
# first_link

# Putting it all together
constructing the structured output from a query.

In [ ]:
import time
from pprint import pprint

user_query="that one example of measuring an eraser"

#documents already uploaded with a modality grounded in text
RAG_response = groundx.search.content(
    id=bucket_id,
    query=user_query
)

retreival_data = []
for res in RAG_response.body['search']['results']:
    suggested_text = res['suggestedText']

    document_data = groundx.documents.get(document_id = res['documentId']).body
    pdf_url = document_data['document']['fileName'].replace(' ','%20')

    example_desc = website_data['standards'][str(document_data['document']['searchData']['example_index'])]['desc']
    google_query = f'archivethecore coherence map {example_desc}'
    first_link = google_search(google_query)

    retreival_data.append({'description': suggested_text, 'pdfUrl': pdf_url, 'exploreUrl':first_link})

llm_response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Describe up to five examples which are most relevent to the query. The output should be a list of objects which include the \"pdfUrl\", \"exploreUrl\", and a brief textual description of why it's relevent, called \"description\". Shorten the description to make it 40 words long max for each example. Make the output in a json format. Do not include markdown formatting around the json object. Only include good examples. If less than five examples are good, you may output fewer. The output should be a list at the top level, not a dict."},
        {"role": "user", "content": f"query: {user_query}\n\n=====example list to be ranked======\n\n{retreival_data}"}
    ]
)
llm_response = llm_response.choices[0].message.content.replace('```json','').replace('```','')
structured_data = json.loads(llm_response)
formatted_output = {'query': user_query, 'response': structured_data}


In [ ]:
pprint(formatted_output)

{'query': 'that one example of measuring an eraser',
 'response': [{'description': 'This resource helps students measure an eraser '
                              'in both centimeters and inches, emphasizing '
                              'understanding different unit sizes and their '
                              'iteration counts.',
               'exploreUrl': 'http://www.google.com/search?q=archive%20the%20core%20coherence%20map%20archivethecore%20coherence%20map%20%3Cp%3EMeasure%20the%20length%20of%20an%20object%20twice%2C%20using%20length%20units%20of%20different%20lengths%20for%20the%20two%20measurements%3B%20describe%20how%20the%20two%20measurements%20relate%20to%20the%20size%20of%20the%20unit%20chosen.%3C/p%3E%0D%0A&btnI',
               'pdfUrl': 'https://achievethecore.org/content/upload/2.MD.A.2_NWEA.pdf'},
              {'description': 'Focuses on measuring and estimating object '
                              'lengths, using a ruler. Supports understanding '
           